In [5]:
import os
import warnings
from dotenv import load_dotenv
warnings.filterwarnings('ignore')
load_dotenv()
api_key = os.environ.get('OPENAI_API_KEY')
if not api_key :
    raise ValueError('OPENAI_API_KEY')

from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [6]:
#script_dir = os.path.dirname(os.path.abspath(__file__)) # abspath 절대경로
# print(script_dir)
# docs_path = os.path.join(script_dir,'sample_docs', 'langraph_rag')
script_dir = os.getcwd()
docs_path = os.path.join(script_dir)
print(f'docs path : {docs_path}')

loader = DirectoryLoader(
    docs_path,
    glob = '**/*.txt',
    loader_cls = TextLoader,
    loader_kwargs={'encoding':'utf-8'},  # 한국어면 꼭 써줄 것
)
document = loader.load()
print(f'읽은 문서의 수 {len(document)}')

docs path : c:\python_src\6.openAI\6
읽은 문서의 수 5


In [7]:
text_splitter = RecursiveCharacterTextSplitter(
     chunk_size=100,
     chunk_overlap=50,
     separators = ['\n\n','\n','.',' ',''],
     length_function=len
)

# 스플릿 = 청킹
doc_splits = text_splitter.split_documents(document)
print(f'청킹개수 : {len(doc_splits)}')

# 임베딩 벡터
embedding_model = OpenAIEmbeddings(model = 'text-embedding-3-small')

청킹개수 : 20


In [ ]:
vectorstroe = Chroma.from_documents(
    documents= doc_splits,
    collection_name='basic_rag_collection',
    embedding=embedding_model
)

retriever = vectorstroe.as_retriever(
    search_type = 'similarity',
    search_kwargs = {'k':3}
)

prompt_template = ChatPromptTemplate.from_messages([
    ('system','Answer question based on the given context in Korean '),
    ('human', 'Context: \n{context}\n\nQuestion: {question}\n\nAnswer:' )])

In [ ]:
def format_docs(docs):
    return '\n\n---\n\n'.join([doc.page_content for doc in docs])

# LCEL 방식 Runnale 객체 실행 invoke - 파이프라인
llm = ChatOpenAI(model = 'gpt-4o-mini',temperature=0)
rag_chain = (                                                                   # 체인은 고정이지만 invoke는 명령어라 부를때마다 변경할 수 있음
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}     # 체인을 여러번쓰려면(다른 질문에도 사용하려면) RunnablePassthrough 가 있어야 함
    |prompt_template
    |llm
    |StrOutputParser()
)

In [ ]:
test_question = [
    'RAG란 무엇인가요',
    'LangGrraph의 핵심 개념을 설명해주세요',
    '프롬프트 엔지니어링 기법에는 어떤 것들이 있나요?'
]
def ask_question(question):
    '''질문에 대한 답변 생성'''
    answer = rag_chain.invoke(question)
    retrieved_docs = retriever.invoke(question)
    sources = [os.path.basename(doc.metadata.get('source', 'unknown')) for doc in retrieved_docs]
    return answer, sources

# 각 질문에 대해 답변 생성
for i,question in  enumerate(test_question,1):
    print(f'question_{i} : {question}')
    answer,sources = ask_question(question)
    print(f'answer : {answer}')
    print(f'sources : {sources}')


question_1 : RAG란 무엇인가요
answer : RAG(검색 증강 생성)는 검색 증강 생성 기술로, 사용자 질문을 임베딩 벡터로 변환한 후, 벡터 데이터베이스에서 유사한 문서를 검색하여, 검색된 문서를 컨텍스트로 사용해 LLM이 답변을 생성하는 방식입니다. RAG의 장점으로는 최신 정보를 반영할 수 있고, 환각(Hallucination)을 감소시키며, 출처를 명시할 수 있는 점이 있습니다.
sources : ['rag_concept.txt', 'rag_concept.txt', 'rag_concept.txt']
question_2 : LangGrraph의 핵심 개념을 설명해주세요
answer : LangGraph의 핵심 개념은 다음과 같습니다:

1. **State(상태)**: 에이전트의 현재 상태를 나타내는 데이터 구조로, 에이전트가 작업을 수행하는 데 필요한 정보를 담고 있습니다.

2. **Node(노드)**: 실제 작업을 수행하는 함수로, 에이전트가 특정 작업을 실행할 때 호출되는 단위입니다.

이러한 개념을 바탕으로 LangGraph는 순환(Cycle)을 지원하여 복잡한 에이전트 워크플로우를 구현할 수 있습니다. LangGraph는 또한 LangChain 위에 구축된 상태 기반 에이전트 프레임워크입니다.
sources : ['langgraph_intro.txt', 'langgraph_intro.txt', 'langgraph_intro.txt']
question_3 : 프롬프트 엔지니어링 기법에는 어떤 것들이 있나요?
answer : 프롬프트 엔지니어링 기법에는 다음과 같은 것들이 있습니다:

1. 프롬프트 템플릿 관리 및 최적화: 효과적인 프롬프트를 만들기 위해 템플릿을 관리하고 최적화하는 기술입니다.
2. 체인(Chains): 여러 구성 요소를 연결하여 파이프라인을 형성하는 기법입니다.
3. 메모리(Memory): 대화의 맥락을 유지하기 위한 메모리 시스템을 활용하는 방법입니다.

이러한 기법들은 LLM에게 명확하고 구체적인 지시를 내리는 